<a href="https://colab.research.google.com/github/ZenBurrito/Predicting-Covid-19-Outbreaks/blob/main/Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Data
import requests
import csv

#EDA and Visualizations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#modeling
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix, mean_squared_error, roc_curve, auc, roc_auc_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import psutil

import datetime



# Business Understanding

Data used in this project came from Kaggle, the CDC, Google Trends, and Twitter. Using basic NLP libraries (NLTK, TweetTokenizer)

For the analysis of the tweets I started with a basic, mostly cleaned, dataset off of Kaggle to create text based predictors. I also gathered my own twitter data about vaccinations, which accounted for another 10000 rows. I used other libraries to clean the text (TextHero), and then used a Trigram CountVectorizer. To create my target variable I used data from Google Trends and the CDC. Using percentages of new cases and trending relevance, I created a score between the two that accounts for not only physical outbreaks, but a social one as well.



Covid-19 is a problem that does not need much introduction or explanantion, however, for this project I wanted to find if there was any correlation between twitter behavior and possible outbreaks. This is important to understand because this could help hospitals better prepare for a response to an outbreak. This could also be applicable to anybody wanting to avoid close contact with the disease, giving them warning beforehand, which they can then use to make more informed decisions on how to handle their day to day.

# Data Preprocessing and Understanding

### Google Trends Data

In [2]:
google = pd.read_csv('/covid_searches.csv')
google

,Category: All categories
Week,Symptoms of COVID-19: (United States)
2020-05-03,35
2020-05-10,34
2020-05-17,32
2020-05-24,30
...,...
2021-12-12,34
2021-12-19,56
2021-12-26,78
2022-01-02,81


In [3]:
google.reset_index(inplace=True)

google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   index                     90 non-null     object
 1   Category: All categories  90 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [4]:
google.drop(index=google.index[0], 
        axis=0, 
        inplace=True)

In [5]:
google['date'] = google['index']
google['relevance'] = google['Category: All categories']
#Renaming to appropriate column names

google.drop(columns='index', inplace=True)
google.drop(columns='Category: All categories', inplace=True)

google

,date,relevance
1,2020-05-03,35
2,2020-05-10,34
3,2020-05-17,32
4,2020-05-24,30
5,2020-05-31,27
...,...,...
85,2021-12-12,34
86,2021-12-19,56
87,2021-12-26,78
88,2022-01-02,81


In [6]:
google['date'] = pd.to_datetime(google['date'])

google['relevance'] = google['relevance'].astype(int)

#Creating True False column for identifying outbreaks
google['outbreak'] = [x for x in google['relevance'] > 75]

In [7]:
google.head(21)

,date,relevance,outbreak
1,2020-05-03,35,False
2,2020-05-10,34,False
3,2020-05-17,32,False
4,2020-05-24,30,False
5,2020-05-31,27,False
6,2020-06-07,32,False
7,2020-06-14,50,False
8,2020-06-21,74,False
9,2020-06-28,88,True
10,2020-07-05,93,True


In [8]:
google.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 1 to 89
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       89 non-null     datetime64[ns]
 1   relevance  89 non-null     int64         
 2   outbreak   89 non-null     bool          
dtypes: bool(1), datetime64[ns](1), int64(1)
memory usage: 2.2 KB


> Data was harvested from Google trends so I could account for the social aspect of outbreaks. A big part of this pandemic has been the fear factor, We are now accutely aware of everything that this pandemic entails, but at the beginning we were woefully unaware of EVERYTHING. So the pandemic came in waves, but in two different ways, the amount of people getting sick and the amount of fear that was placed into the public. Whereas accounting for the google search relevance doesnt necessarily give us how scared we were at the time, it still does tell us when it was trending upwards.

### CDC Data

In [9]:
CDC = pd.read_csv('/USA_covid_stats.csv')
CDC

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0,03/12/2021 03:20:13 PM,Agree,NaN
1,06/11/2021,TX,2965966,NaN,NaN,1463,355.0,51158,NaN,NaN,17,0.0,06/13/2021 12:00:00 AM,Not agree,Not agree
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0,01/03/2022 03:18:16 PM,NaN,NaN
3,08/22/2020,AR,56199,NaN,NaN,547,0.0,674,NaN,NaN,11,0.0,08/23/2020 02:15:28 PM,Not agree,Not agree
4,07/17/2020,MP,37,37.0,0.0,1,0.0,2,2.0,0.0,0,0.0,07/19/2020 12:00:00 AM,Agree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43135,05/28/2020,IA,18585,NaN,NaN,228,0.0,506,NaN,NaN,14,0.0,05/29/2020 02:19:55 PM,Not agree,Not agree
43136,06/07/2020,SD,5438,NaN,NaN,71,0.0,65,64.0,1.0,0,1.0,06/08/2020 02:55:08 PM,NaN,Agree
43137,04/30/2021,SD,122660,NaN,NaN,128,17.0,1967,1601.0,366.0,5,1.0,05/01/2021 01:43:22 PM,NaN,Agree
43138,03/10/2021,SD,113962,NaN,NaN,209,37.0,1904,1546.0,358.0,3,0.0,03/11/2021 03:36:21 PM,NaN,Agree


In [10]:
CDC['date'] = CDC['submission_date']

CDC.drop(columns='submission_date', axis=0, inplace=True)

In [11]:
col = ['state', 'conf_cases', 'prob_cases', 'pnew_case', 'conf_death', 'prob_death', 'pnew_death', 'created_at', 'consent_cases', 'consent_deaths', 'new_case', 'tot_death', 'new_death']

CDC.drop(columns=col, inplace=True)

In [12]:
CDC.dropna(how='all', inplace=True)

In [13]:
CDC['date'] = pd.to_datetime(CDC['date']).dt.normalize()

In [14]:
CDC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43140 entries, 0 to 43139
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   tot_cases  43140 non-null  int64         
 1   date       43140 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 1011.1 KB


In [15]:
CDC.isna().sum()

tot_cases    0
date         0
dtype: int64

In [16]:
CDC = CDC.groupby(['tot_cases', pd.Grouper(key='date', freq='W-SUN')]).sum().reset_index().sort_values('date')
CDC

,tot_cases,date
0,0,2020-01-26
87,1,2020-01-26
1,0,2020-02-02
88,1,2020-02-02
128,2,2020-02-02
...,...,...
11827,68013,2022-01-09
29507,800743,2022-01-09
18648,187467,2022-01-09
32020,1101111,2022-01-09


> Putting political issues aside, between the WHO and the CDC, the most accurate information about covid-19 comes from them. So in order to construct a score for outbreaks, I went through the CDC API to gather actual US covid statistics. This accounts for the physical aspect of outbreaks. There have been several scenarios where outbreaks have been quite obvious. The beginning spike, the Delta variant, and most recently Omicron.

### Twitter Data

In [17]:
tweets = pd.read_csv('/covid_tweets.csv')
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


> After an intitial look over the Twitter data I can already observe it's very messy, The only data I want from it currently is the text data and the converted datetime date column.

In [18]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179108 entries, 0 to 179107
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         179108 non-null  object
 1   user_location     142337 non-null  object
 2   user_description  168822 non-null  object
 3   user_created      179108 non-null  object
 4   user_followers    179108 non-null  int64 
 5   user_friends      179108 non-null  int64 
 6   user_favourites   179108 non-null  int64 
 7   user_verified     179108 non-null  bool  
 8   date              179108 non-null  object
 9   text              179108 non-null  object
 10  hashtags          127774 non-null  object
 11  source            179031 non-null  object
 12  is_retweet        179108 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 15.4+ MB


In [19]:
column = ['user_name','user_location','user_description','user_created','user_followers','hashtags', 'user_favourites','user_verified','is_retweet', 'source', 'user_friends']

tweets.drop(columns=column, inplace=True)

In [20]:
tweets['date'] = pd.DatetimeIndex(tweets['date']).normalize()

In [21]:
tweets = tweets.groupby(['text', pd.Grouper(key='date', freq='W-SUN')]).sum().reset_index().sort_values('date')

> Grouping the date by week so I am not overloaded by columns, also so the target variable is by week

In [22]:
tweets

,text,date
108519,Join HealthCube's Ramanan Laxminarayan alongsi...,2020-07-26
26897,100 and 40 bedded field hospitals established ...,2020-07-26
75618,Cures/vaccines for #covid19\n1Mangaluru congre...,2020-07-26
75627,Curious Times Weekly 📰#Space🎑 #Mars #forks in ...,2020-07-26
75628,Curious about #COVID19? Expert pathologist @Cl...,2020-07-26
...,...,...
71261,"Cases: 25,009,250 (+11,923) \nDeaths: 842,702 ...",2020-08-30
71262,"Cases: 25,009,739 (+489) \nDeaths: 842,709 (+7...",2020-08-30
71263,"Cases: 25,015,362 (+5,623) \nDeaths: 842,778 (...",2020-08-30
118089,MzansiReader App. Link in the bio to download ...,2020-08-30


In [23]:
tweets.date.value_counts()

2020-08-16    42340
2020-08-02    42210
2020-08-09    30738
2020-08-23    26527
2020-07-26    24633
2020-08-30    12446
Name: date, dtype: int64

In [24]:
tweets.isna().sum()

text    0
date    0
dtype: int64

In [27]:
!pip install texthero

     |████████████████████████████████| 235 kB 9.4 MB/s 
     |████████████████████████████████| 1.5 MB 45.3 MB/s 
     |████████████████████████████████| 748 kB 23.9 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [28]:
import texthero as hero

tweets['text'] = hero.clean(tweets['text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


> Found a library that is used for cleaning up text data pretty proficiently, called TextHero

In [29]:
vax_tw = pd.read_csv('/vaccination_tweets.csv')
vax_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11020 entries, 0 to 11019
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                11020 non-null  int64 
 1   user_name         11020 non-null  object
 2   user_location     8750 non-null   object
 3   user_description  10341 non-null  object
 4   user_created      11020 non-null  object
 5   user_followers    11020 non-null  int64 
 6   user_friends      11020 non-null  int64 
 7   user_favourites   11020 non-null  int64 
 8   user_verified     11020 non-null  bool  
 9   date              11020 non-null  object
 10  text              11020 non-null  object
 11  hashtags          8438 non-null   object
 12  source            11019 non-null  object
 13  retweets          11020 non-null  int64 
 14  favorites         11020 non-null  int64 
 15  is_retweet        11020 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 1.2+ MB


In [30]:
cols = ['id','user_name','user_location','user_description','user_created','user_followers','user_favourites','user_verified','hashtags','source','retweets','favorites','is_retweet', 'user_friends']
vax_tw.drop(columns=cols, inplace=True)
#Dropping irrelevant columns for project


In [31]:
vax_tw['date'] = pd.DatetimeIndex(vax_tw['date']).normalize()
#Converting to Datetime to join by later

vax_tw = vax_tw.groupby(['text', pd.Grouper(key='date', freq='W-SUN')]).sum().reset_index().sort_values('date')
#Grouping by week to match the rest of the datasets

In [32]:
vax_tw['text'] = hero.clean(vax_tw['text'])
#Using Aforementioned library to clean data

vax_tw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11014 entries, 7007 to 1927
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   text    11014 non-null  object        
 1   date    11014 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 258.1+ KB


> Here I combined two different Twitter datasets. Both datasets are mostly about covid-19 and vaccinations. I pulled from relevant subjects because I believed that they would be the most correlated with predicting a pandemic. There are some normal tweets imbedded within that serves as an almost baseline.

> To review, My data came from multiple sources, First I started with a base, almost cleaned, Kaggle dataset that had about 170k rows of tweets, for the sake of time and computational effectiveness, I trimmed down the dataset so I had more time for model tuning and other aspects of the project. I also gathered another 11k rows of tweets that were about vaccinations. Those two datasets were both used to create text based predictors using a CountVectorizer, to see if there was any direct correlation between the tweets and my target variable.

>The target variable was constructed with both data from Google and the CDC. I created a score using the percentages of new cases as well as the relevance of searches. I then scaled the score down so outliers did not effect them as much. This accounts for not only physical outbreaks, but the social aspect as well.

## Combining Dataframes

In [33]:
tweet = tweets.sample(frac=0.50)
#Fractioned out so I can test more ideas frequently
#Also fractioned due to memory constraints

df = [tweet, vax_tw, google]

> Have to fraction out data due to computational effectiveness

In [34]:
from functools import reduce

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['date'],
                                            how='outer'), df)

> Outer join is necessary due to data being dropped with inner join on date

In [35]:
df_merged

,text_x,date,text_y,relevance,outbreak
0,till hits family member ' real https co arwpln...,2020-08-09,NaN,53,False
1,covid19 crap sods everyone sex beach ways,2020-08-09,NaN,53,False
2,chart day jpediatr barbara chaiyachati amp chi...,2020-08-09,NaN,53,False
3,japan covid19 mutating tokyo olympics may need...,2020-08-09,NaN,53,False
4,new cases deaths reported india last hours lak...,2020-08-09,NaN,53,False
...,...,...,...,...,...
100492,NaN,2021-12-12,NaN,34,False
100493,NaN,2021-12-19,NaN,56,False
100494,NaN,2021-12-26,NaN,78,True
100495,NaN,2022-01-02,NaN,81,True


In [36]:
df_merged['text_x'].fillna(df_merged['text_y'], inplace=True)

In [37]:
df_merged.drop(columns='text_y', inplace=True)

In [38]:
df_merged.text_x.drop_duplicates(inplace=True)

In [39]:
df_merged.dropna(inplace=True)

In [40]:
df_merged.date.value_counts()

2020-08-02    21253
2020-08-16    20903
2020-08-09    15320
2020-08-23    13420
2020-07-26    12256
2020-08-30     6295
2021-01-10      867
2020-12-20      860
2021-01-17      792
2020-12-27      642
2021-01-24      595
2021-01-03      551
2021-01-31      535
2021-02-28      414
2021-02-21      354
2021-02-07      328
2021-08-29      324
2021-04-04      294
2021-03-21      268
2021-03-14      264
2021-03-07      263
2021-05-30      241
2021-02-14      240
2021-03-28      227
2021-04-18      203
2021-05-23      177
2021-04-11      175
2021-06-20      163
2020-12-13      161
2021-07-18      156
2021-07-25      151
2021-08-22      144
2021-08-15      140
2021-08-01      133
2021-08-08      120
2021-10-10      119
2021-07-11      113
2021-09-26      107
2021-05-16      102
2021-09-12       99
2021-09-05       97
2021-10-03       93
2021-06-27       90
2021-11-14       57
2021-11-07       55
2021-09-19       49
2021-11-21       46
2021-04-25       43
2021-06-13       43
2021-10-24       40


In [41]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100461 entries, 0 to 100460
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   text_x     100461 non-null  object        
 1   date       100461 non-null  datetime64[ns]
 2   relevance  100461 non-null  int64         
 3   outbreak   100461 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(1)
memory usage: 3.2+ MB


> After joining together all of my dataframes, I realized there was a major problem, the only join that was functional was an outer join. When I attempted an inner join on the date columns they were deleting the values within and leaving only column names. To combat this I did an outer join with my dataframes, but had to fraction my larger twitter dataframe due to computational effectiveness.

## Functions

In [42]:
def tokenize(tweet):
    tknzr = TweetTokenizer(strip_handles=True, reduce_len=True, 
                           preserve_case=False)
    return tknzr.tokenize(tweet)

In [43]:
def classifiers(x_train, y_train):
    nb  = MultinomialNB()
    nb.fit(x_train, y_train)
    log = LogisticRegression()
    log.fit(x_train, y_train)
    forest = RandomForestClassifier(n_estimators=100, max_depth=5)
    forest.fit(x_train, y_train)
    gradboost = GradientBoostingClassifier(random_state=123, max_depth=5, 
                                          learning_rate = 0.01)
    gradboost.fit(x_train, y_train)
    adaboost = AdaBoostClassifier(n_estimators=100)
    adaboost.fit(x_train, y_train)
    svm = SVC(kernel='linear', probability=True)
    svm.fit(x_train, y_train)
    return [nb, log, forest, gradboost, adaboost, svm]


In [44]:
def classifier_performance(vectorizer, train_data, test_data, y_test):
    accuracy_df = []
    x_train = vectorizer.fit_transform(train_data)
    x_test = vectorizer.transform(test_data)
    
    classifier_list = classifiers(x_train, y_train)
    for i in classifier_list:
        preds = i.predict(x_test)
        accuracy = accuracy_score(y_test, preds)
  
    accuracy_df = pd.DataFrame(accuracy_df)
    classifiers_key = ['Naive Bayes', 'Random Forest', 'Gradient Boost',
                      'AdaBoost', 'Support Vector Machine']
    accuracy_df['Model'] = classifiers_key
    accuracy_df.rename(columns={0: 'Accuracy'}, inplace=True)
    fin_accuracy_df = accuracy_df[['Model', 'Accuracy']]
    return fin_accuracy_df

In [45]:
def model_performance(model, vectorizer):
  
    train_data = vectorizer.fit_transform(x_train)
    test_data = vectorizer.transform(x_test)

    model.fit(train_data, y_train)

    pred = model.predict(test_data)

    return f'{model}', accuracy_score(y_test, pred)

In [46]:
def griddy(model, params, vectorizer):
  X_train = vectorizer.fit_transform(x_train)
  X_test = vectorizer.transform(x_test)

  grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=6)
  grid.fit(X_train, y_train)

  return [grid.best_params_, grid.accuracy_score, grid.recall_score]

> Functions used throughout the project, old functions are still in here from my previous process, and working through the project.

## Count Vectorization

> In this next section I created baseline models to run through my data on different vectorizations. I used a Count Vectorizer and TFIDF Vectorizer with different n-grams to test the potential of the models.

>After taking an iterative modeling approach, I found that one model performed exceptionally better than other models. Using the Multinomial Naive Bayes model I was able to predict outbreaks at a 40 percent accuracy. Part of the reason the Naive Bayes worked better than other models is because it assumes independence between my variables, meaning each of them is counted on its own without any weights, this pairs well with a CountVectorizer because it establishes importance based on word count.

In [53]:
data = df_merged['text_x']
labels = df_merged['relevance']
x_train, x_test, y_train, y_test = train_test_split(data, labels)
tfidfvec = TfidfVectorizer(stop_words='english', tokenizer=tokenize)
tfidfvec2 = TfidfVectorizer(stop_words='english', tokenizer=tokenize, ngram_range=(1,2))
tfidfvec3 = TfidfVectorizer(stop_words='english', tokenizer=tokenize, ngram_range=(1,3))
countvec = CountVectorizer(stop_words='english', tokenizer=tokenize)
countvec2 = CountVectorizer(stop_words='english', tokenizer=tokenize, ngram_range=(1,2))
countvec3 = CountVectorizer(stop_words='english', tokenizer=tokenize, ngram_range=(1,3))

In [54]:
nb  = MultinomialNB()

In [55]:
model_performance(nb, countvec)

('MultinomialNB()', 0.2979375696767001)

In [56]:
forest = RandomForestClassifier(n_estimators=100, max_depth=5)


In [57]:
model_performance(forest, countvec)

('RandomForestClassifier(max_depth=5)', 0.22790253225035834)

In [58]:
#gradboost = GradientBoostingClassifier(random_state=123, max_depth=5, learning_rate = 0.01)
#model_performance(gradboost, countvec)

In [59]:
#adaboost = AdaBoostClassifier(n_estimators=100)
#model_performance(adaboost, countvec)

### Bigram

In [60]:
model_performance(nb, countvec2)

('MultinomialNB()', 0.31692944736422995)

In [61]:
model_performance(forest, countvec2)

('RandomForestClassifier(max_depth=5)', 0.2260710304188565)

In [62]:
#model_performance(gradboost, countvec2)

In [63]:
#model_performance(adaboost, countvec2)

### Trigram

In [64]:
model_performance(nb, countvec3)

('MultinomialNB()', 0.3192785475394171)

In [65]:
model_performance(forest, countvec3)

('RandomForestClassifier(max_depth=5)', 0.22260710304188566)

In [66]:
#model_performance(gradboost, countvec3)

In [67]:
#model_performance(adaboost, countvec3)

## TF-IDF Vectorization



### Unigram

In [68]:
model_performance(nb, tfidfvec)

('MultinomialNB()', 0.2883420926899188)

In [69]:
model_performance(forest, tfidfvec)

('RandomForestClassifier(max_depth=5)', 0.2273053033922599)

In [70]:
#model_performance(gradboost, tfidfvec)

In [71]:
#model_performance(adaboost, tfidfvec)

### Bigram

In [72]:
model_performance(nb, tfidfvec2)

('MultinomialNB()', 0.29777830864787386)

In [73]:
model_performance(forest, tfidfvec2)

('RandomForestClassifier(max_depth=5)', 0.22782290173594522)

In [74]:
#model_performance(gradboost, tfidfvec2)

In [75]:
#model_performance(adaboost, tfidfvec2)

### Trigram

In [76]:
model_performance(nb, tfidfvec3)

('MultinomialNB()', 0.29606625258799174)

In [77]:
model_performance(forest, tfidfvec3)

('RandomForestClassifier(max_depth=5)', 0.22419971333014813)

In [78]:
#model_performance(gradboost, tfidfvec3)

In [79]:
#model_performance(adaboost, tfidfvec3)

All of the gradboost and adaboost models were commented out due to taking FOREVER to run, so in order to get a final notebook with outputs ran in, I decided to let the work be shown, but the models not ran due to the time and poor scores they provide.

## Improving models

> Trying to improve the best performing models from the previous cells, The two best performing were Random Forests and Naive Bayes, So running grid searches on them to see if there is any hyperparameter tuning that I could do to further improve the models.


In [80]:
X_train = countvec3.fit_transform(x_train)
X_test = countvec3.transform(x_test)

param_grid = {'alpha': [1, 5, 10, 50, 100]}

clf = GridSearchCV(estimator=nb, param_grid=param_grid, n_jobs=-1, cv=6, scoring='accuracy')
clf.fit(X_train, y_train)



GridSearchCV(cv=6, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [1, 5, 10, 50, 100]}, scoring='accuracy')

In [81]:
clf.best_params_

{'alpha': 1}

In [ ]:
params = {'criterion' : ['gini', 'entropy'],
          'max_depth' : [3, 4, 5, 8],
          'min_samples_split' : [2, 4, 10],
          'min_samples_leaf' : [1, 2, 4, 5],
          'n_estimators' : [10, 50, 100, 200],
          'bootstrap' : [True, False]}

griddy(forest, params, countvec3)